<br>
# **RNN 과 LSTM**
자연어에서 딥러닝의 활용 : Machine Learning for Natural Language

<br></br>
## **1 CNN**

Text 분류를 CNN을 사용하여 훈련모델을 생성 [Blog](http://docs.likejazz.com/cnn-text-classification-tf/)

1. <strong>Convolution Neural Network</strong> : 2D convolution 예제
1. <strong>Convolution Layer : 합성곱 계층</strong>으로 가중치와 편향을 적용한다
1. <strong>Pooling Layer</strong> : 단순히 최적의 값을 선택하여 가져온다
1. <strong>Kernel</strong> : 3x3 <strong>Kernel</strong>을, 1 <strong>stride</strong> 씩 이동하며 <strong>Convolution layer</strong>를 생성


<img src="http://www.wildml.com/wp-content/uploads/2015/11/Screen-Shot-2015-11-06-at-8.03.47-AM-1024x413.png" align="left" width='800'>

<br></br>
## **2 LSTM - RNN을 사용한 단어 자동완성 모델 학습하기**
1. 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성한다.
1. Basic RNN과 비교하여, Cell 거리에 따른 비중을 반영하여 (tan h) 모델을 생성한다

1. <strong>RNN Cell 3개 : 128개 신경망, Drop Out, 128개 신경망</strong>
1. 자연어 처리나 음성 처리 분야에 많이 사용
1. 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성

<img src="https://www.kdnuggets.com/wp-content/uploads/RNN.jpg" align="left" width='400'>

<img src="https://kakalabblog.files.wordpress.com/2017/04/c968d4ad-390d-4235-9144-97572181c9f9.png">

<br></br>
### **01 알파벳을 기본으로 학습 데이터 정의**
텍스트의 알파벳을 One-hot encoding 으로 세분화된 모델 만들기

In [1]:
import tensorflow as tf
import numpy as np

# rar 알파벳 목록을 One-Hot 인코딩 배열생성 : {'a':0, 'b':1, 'c':2, ... ,}
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
num_dic  = {n: i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

In [2]:
# 알파벳 배열을 인덱스 번호로 변환 : input_batch, target_batch 에 적용 
# [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
# np.eye() 대각성분은 '1', 나머지 '0'인 항등행렬/ 단위행렬을 생성한다
def make_batch(seq_data):
    input_batch, target_batch = [], []
    # 단어의 알파벳을 one-hot을 batch 데이터 생성
    for seq in seq_data: 
        input_num = [num_dic[n] for n in seq[:-1]]  # 3, 3, 15, 4, 3 ...
        target    = num_dic[seq[-1]]                
        input_batch.append(np.eye(dic_len)[input_num]) # 출력 1 : one-hot 인코딩
        target_batch.append(target)                    # 출력 2 : one-hot 인덱스값
    return input_batch, target_batch

<br></br>
### **02 LSTM 모델을 정의하기**
1. Tensorflow 파라미터 정의하기
1. LSTM 모델 Graph 정의하기
1. Cost, 최적화 함수 정의하기

In [3]:
# 1. RNN 신경망 모델 정의
# (1) 파라미터 설정 : RNN 시퀀스 갯수  ex) 타입 스텝: [1 2 3] => 3
tf.reset_default_graph()          # tf 매개변수 초기화 함수 
learning_rate = 0.01
n_step = 3                        # 전체 단어중 처음 3글자를 단계적 학습
n_hidden, total_epoch = 64, 30
n_input = n_class = dic_len       # 출력, 입력값 모두 26 (알파벳 one-hot을 사용)

In [4]:
# (2) 알파벳 one-hot 인코딩으로 26개 값으로 출력
# cost함수 : sparse_softmax_cross_entropy_with_logits 는 인덱스를 그대로 사용
# if) one-hot 인코딩 사용 시 : ( tf.float23, [None, n_class] )
# 1번 학습시 batch 에 해당하는 크기의 데이터를 사용한다 
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32,   [None])  # one-hot 의 인덱스데이터 사용 ex) [2], [12] ...

W = tf.Variable(tf.random_normal([n_hidden, n_class])) #, reuse=True)
b = tf.Variable(tf.random_normal([n_class]))           #, reuse=True)  #https://github.com/tensorflow/tensorflow/issues/799

In [5]:
# (3) RNN 셀의 정의
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합을 방지하기 위하여 Dropout을 설정
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)    
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)         
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2]) # Cell1, Cell2를 조합한 RNN 셀

In [6]:
# (4) 최종 결과는 one-hot 인코딩 형식으로 변환
# tf.nn.dynamic_rnn 를 이용해 순환 신경망(Deep RNN)을 생성 (time_major = True)
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model   = tf.matmul(outputs, W) + b

In [7]:
# softmax_cross_entropy_with_logits : 텍스트 데이터 one-hot의 index 를 받는다
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br></br>
### **03 모델을 Train 하기**
1. 학습 데이터를 입력
1. Graph 모델을 Tensorflow 의 Session을 활용하여 학습하기

In [8]:
# Train 데이터를 정의한다
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [9]:
# 2. RNN 신경망 학습
sess = tf.Session()

sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
# save_file = './data/word_model.ckpt'
# saver = tf.train.Saver(tf.global_variables())
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    if epoch % 4 == 0:
        print('Epoch: {:.4f} cost = {:.6f}'.format(epoch + 1, loss))
# saver.save(sess, save_file)
print('최적화 완료!')

Epoch: 1.0000 cost = 3.670792
Epoch: 5.0000 cost = 1.703650
Epoch: 9.0000 cost = 0.649892
Epoch: 13.0000 cost = 0.582221
Epoch: 17.0000 cost = 0.440267
Epoch: 21.0000 cost = 0.229660
Epoch: 25.0000 cost = 0.204612
Epoch: 29.0000 cost = 0.195957
최적화 완료!


<br></br>
### **04 학습한 모델의 성능확인 및 평가**
예측 데이터를 활용하여 모델의 정확성 판단하기

In [10]:
# 3. 모델의 성능을 평가 (레이블이 one-hot 인덱스 정수이므로, 예측값도 정수로 변경)
# tf.cast(x, dtype, name=None) : 텐서를 새로운 자료형으로 변환
prediction       = tf.cast(tf.argmax(model, 1), tf.int32) # one-hot의 인덱스 추출
prediction_check = tf.equal(prediction, Y)                # 입력값을 Target 과 비교한다
accuracy         = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val     = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

In [11]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:2] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:-2] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)
sess.close()


=== 예측 결과 ===
입력값: ['wo ', 'wo ', 'de ', 'di ', 'co ', 'co ', 'lo ', 'lo ', 'ki ', 'ki ']
예측값: ['wod', 'wod', 'dep', 'die', 'cod', 'cod', 'lod', 'loe', 'kis', 'kid']
정확도: 0.9


<br>
## **3 Seq 2 Seq : 구글의 기계번역에 사용하는 신경망 모델**
Seq2Seq는 구글 기계번역에 사용하는 신경망 모델이다 (http://aidev.co.kr/deeplearning/2273)
1. LSTM 셀을 여러개 합쳐진 모델을 활용
1. GAN 의 Encoder 와 Decoder 를 합친 학습모델을 만드는데 유용하다
1. LSTM 보다 위상값 차이를 더 확실하게 학습이 가능하여, 문장의 학습 및 출력용 모델을 만드는데 적합하다
1. 입력 RNN과 출력 신경망의 조합한 모델로써, 입력된 문장과 출력된 문장간의 학습시 사용된다
1. LSTM을 encoder 와 Decode로 분리하여 단위 뉴런의 길이가 더 길고 위상값이 서로다른 다른 모델을 학습한다
1. [챗봇 블로그](http://yujuwon.tistory.com/entry/TENSORFLOW-seq2seq-%EA%B8%B0%EB%B0%98-%EC%B1%97%EB%B4%87-%EB%A7%8C%EB%93%A4%EA%B8%B0)
1. <strong>'RNN 인코더 셀'</strong> (A, B, C) 과 <strong>'RNN 디코더 셀'</strong> (W, X, Y, X) RNN 2개 모델을 결합한다
1. 기본모델에 <strong>특수함 심벌</strong> ((1) 입력의 시작을 알림 (2) 디코더 출력이 끝남을 알림)을 포함한다 
1. <strong>'챗봇', '번역', '이미지 캡셔닝'</strong> 등의 모델을 생성한다
1. 문장등을 입력받은 뒤 <strong>문장의 번역</strong>등을 출력한다

<img src="https://t1.daumcdn.net/cfile/tistory/24FA50335965BB032F?original" align="left" width="800">

<br>
### **01 모델의 정의**
1. Data, Cell, Graph를 정의하고
1. Cost 함수, Optimze 함수를 선택한다

In [ ]:
# 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어봅니다.
import tensorflow as tf
import numpy as np

In [12]:
# 1. Input Data를 설정한다
# (1) elements 를 정의한 뒤 (알파벳과 한글), {dict} 데이터 형식으로 변환

# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd'] ||  to   -> ['t', 'o', 'P', 'P']
char_arr = [ c for c in 'SPabcdefghijklmnopqrstuvwxyz나놀녀단랑무사소스어이키E'] 
num_dic  = { n : i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

In [13]:
# (2) 번역학습 데이터 입력
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], 
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

# (3) 데이터를 One-Hot-Encoding 으로 변환
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]           # 인코더 셀 입력값 : 입력단어를 한글자씩 떼어 배열생성
        output = [num_dic[n] for n in ('S' + seq[1])]  # 디코더 셀 입력값 : 시작을 나타내는 'S'심볼을 맨 앞에 붙인다
        target = [num_dic[n] for n in (seq[1] + 'E')]  # 디코더 셀 출력값 : 비교값으로, 끝을 알리기 위해 끝에 'E'를 붙인다
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아니다 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)
    return input_batch, output_batch, target_batch

In [14]:
# 2. 모델의 정의
# (1) 파라미터값 설정
tf.reset_default_graph()      # tf 매개변수를 초기화 
learning_rate = 0.01
n_hidden, total_epoch = 128, 100
n_class = n_input = dic_len  # 입력과 출력 : one-hot 인코딩으로 크기가 같다

# (2) 인코더/ 디코더 입 출력 설정
# Seq2Seq은 '인코더 입력'과 '디코더 입력'의 형식이 같다 : [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])  # [batch size, time steps]

In [15]:
# (3) 인코더/ 디코더 셀 설정
# 1) tf.variable_scope() : 인코더 셀을 정의한다
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = 0.5) 
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,dtype=tf.float32)

In [16]:
# 2) 디코더 셀을 정의한다
# '인코더 셀'의 '최종 상태값'을, '디코더 셀'의 '초기 상태값'과 연결한다
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state = enc_states, 
                                            dtype = tf.float32)

In [17]:
# (4) 설정한 매개변수와 Cell을 연결하여 Seq2Seq 모델을 생성한다
model = tf.layers.dense(outputs, n_class, activation=None)
cost  = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br>
### **02 모델의 학습**
Train by the TensorFlow

In [18]:
# 3. 모델의 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 9 == 0 :
        print('Epoch: {:4d}  cost = {:.6f}'.format((epoch + 1),loss))
print('최적화 완료!')

Epoch:    1  cost = 3.743667
Epoch:   10  cost = 0.093628
Epoch:   19  cost = 0.019045
Epoch:   28  cost = 0.005433
Epoch:   37  cost = 0.001452
Epoch:   46  cost = 0.001286
Epoch:   55  cost = 0.001034
Epoch:   64  cost = 0.000333
Epoch:   73  cost = 0.000132
Epoch:   82  cost = 0.000345
Epoch:   91  cost = 0.000243
Epoch:  100  cost = 0.000602
최적화 완료!


<br>
### **03 학습 모델의 활용**
Train by the TensorFlow

In [19]:
# (1) 예측시 사전에 없는 단어는 'P'로 채운다. ex) ['word', 'PPPP']
# (2) [batch size, time step, input] 를 예측결과로 출력 : 2번째 차원값 input차원의 argmax로 예측한다
def translate(word):
    seq_data = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded = [char_arr[i] for i in result[0]]  # 결과 값 '인덱스'에 해당글자를 사용하여 배열을 만든다.
    end = decoded.index('E')                    # 출력끝 을 의미하는 'E' 이후의 글자들은 제거한다
    translated = ''.join(decoded[:end])
    return translated

print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
word -> 단어
wodr -> 단무
love -> 사랑
loev -> 사랑
abcd -> 소녀


### 02.ml_for_nltk
word2Vec
1. Gensim
1. Tensorflow
1. tSNE

RNN
1. 삼행시 봇

LSTM 문장생성
1. HMM 은 유사도만 측정 가능하고, 그 확률이 낮아서 문장생성은 어렵다
1. LSTM도 낮은 확률에 근거하여 Text를 생성하지만, 반복 학습을 통해서 오차를 줄이는 과정을 통해 정밀성을 높일 수 있다

seq2seq 챗봇
1. Text 생성
1. 어린왕자 Text를 대통령 연설문으로 바꿔서 챗봇 구현하기

Attention
1. Image Detection